In [2]:
import numpy as np
import math

In [7]:
def update_df(R:float , c , d):
    f = np.int64(R) 
    z= c %32
    if z!=0:
        d = (c << (d%z))^d
    else:
        d= d^c
    d = d^ (d<<21)
    d = d ^ (d>>35)
    d  =d ^ (d<<4)
    
    return [f ,d]

In [8]:
update_df(2.345647768 , 157 , 524592858)

[np.int64(2), 332107477252159304]

In [ ]:
# ---- Main: Substitute function (forward + backward) ----
def Substitute(B):
    """
    Forward and backward substitution for one image block.
    B: list/array of pixel values [b1, b2, ..., bn]
    Returns: list of final substituted pixels (out)
    """
    n = len(B)
    out1 = []

    # --- Forward pass ---
    f = np.int64(1)  # initial f (non-zero)
    d = np.int64(1)  # initial d (non-zero)

    for bi in B:
        R = 17.32 * math.sqrt(abs(d) / (4 * abs(f) + 1e-9))

        k = int(R) % 256
        si = k ^ bi
        out1.append(si)
        f, d = update_df(R, bi, d)

    # --- Backward pass ---
    out = []
    f = np.int64(1)  # re-initialize f
    d = np.int64(1)  # re-initialize d

    for si in reversed(out1):  # process from right to left
        R = 17.32 * math.sqrt(abs(d) / (4 * abs(f) + 1e-9))

        k = int(R) % 256
        ci = k ^ si
        out.insert(0, ci)  # prepend to maintain original order
        f, d = update_df(R, si, d)

    return out
block = [120, 45, 200, 88]

output = Substitute(block)
print("Input Block: ", block)
print("Output Block:", output)

Input Block:  [120, 45, 200, 88]
Output Block: [112, 106, 70, 37]


In [ ]:
def Substitute_Inv(out):
    """
    Inverse substitution for decryption.
    out: encrypted block [c1, c2, ..., cn]
    Returns: original block B
    """
    n = len(out)
    out1 = []

    # --- Backward pass first ---
    f = np.int64(1)
    d = np.int64(1)

    for ci in reversed(out):  # i = n down to 1
        R = 17.32 * math.sqrt(d / (4 * f))
        k = int(R) % 256
        si = k ^ ci
        out1.insert(0, si)  # prepend to maintain order
        f, d = update_df(R, si, d)

    # --- Forward pass second ---
    B = []
    f = np.int64(1)  # re-initialize
    d = np.int64(1)

    for si in out1:  # i = 1 to n
        R = 17.32 * math.sqrt(d / (4 * f))
        k = int(R) % 256
        bi = k ^ si
        B.append(bi)
        f, d = update_df(R, bi, d)

    return B
